# Introduction to generative adversarial networks

In [74]:
import keras
from keras import layers
import numpy as np

latent_dim = 32
height = 32
width = 32
channels = 3




## Le générateur  
Tout d'abord, développons un modèle générateur qui transforme un vecteur (de l'espace latent-
pendant la formation, il sera échantillonné de manière aléatoire) en une image candidate. L'un des
l'un des nombreux problèmes qui se posent couramment avec les GAN est que le générateur reste bloqué avec des images générées qui ressemblent à du bruit.
images générées qui ressemblent à du bruit. Une solution possible est d'utiliser le dropout à la fois sur le dis- criminateur et le générateur.
criminateur et le générateur.
Listing 8.29 Générateur GAN


In [75]:
generator_input = keras.Input(shape=(latent_dim,))
x = layers.Dense(128 * 16 * 16)(generator_input)
#x = layers.Dense(64 * 64 * 64)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)
#x = layers.Reshape((32, 32, 128))(x)

# Transforms the input into a 16 × 16 128-channel feature map




x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

x= layers.Conv2D(256, 5, padding='same')(x)
x= layers.LeakyReLU()(x)
x= layers.Conv2D(256, 5, padding='same')(x)
x= layers.LeakyReLU()(x)

#Upsamples to 32 × 32
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)

generator = keras.models.Model(generator_input, x)
generator.summary()


Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_22 (InputLayer)        [(None, 32)]              0         
_________________________________________________________________
dense_14 (Dense)             (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_59 (LeakyReLU)   (None, 32768)             0         
_________________________________________________________________
reshape_7 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_52 (Conv2D)           (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_60 (LeakyReLU)   (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose_6 (Conv2DTr (None, 32, 32, 256)       104

## Le discriminateur
Ensuite, vous allez développer un modèle de discriminateur qui prend en entrée une image candidate
(réelle ou synthétique) et la classe dans l'une des deux catégories suivantes : "image générée" ou "image réelle
image réelle provenant de l'ensemble d'apprentissage ".
Listing 8.30 Le réseau discriminant GAN

In [76]:
discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)


x = layers.LeakyReLU()(x)

x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)

x = layers.Dropout(0.4)(x) #One dropout layer: an important trick!
x = layers.Dense(1, activation='sigmoid')(x) # Classification layer

discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()
discriminator_optimizer = keras.optimizers.RMSprop(
    lr=0.0008,
    clipvalue=1.0,
    decay=1e-8)

discriminator.compile(optimizer=discriminator_optimizer,
loss='binary_crossentropy')

Model: "model_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_23 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_64 (LeakyReLU)   (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_65 (LeakyReLU)   (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_66 (LeakyReLU)   (None, 6, 6, 128)         0  

In [77]:
height

32

# Le réseau adversatif
Enfin, vous allez configurer le GAN , qui enchaîne le générateur et le discriminateur.
Une fois entraîné, ce modèle va déplacer le générateur dans une direction qui améliore sa capacité à tromper le discriminateur.
à tromper le discriminateur. Ce modèle transforme les points de l'espace latent en une décision de classification - "faux" ou "vrai".
en une décision de classification - "faux" ou "vrai" - et il est censé être entraîné avec des étiquettes qui sont toujours "ces images sont réelles".
"ce sont de vraies images". Donc, l'entraînement de gan va mettre à jour les poids du générateur d'une manière
qui rend le discriminateur plus susceptible de prédire "réel" lorsqu'il regarde des images fausses.
Il est très important de noter que vous avez configuré le discriminateur pour qu'il soit gelé pendant l'entraînement (non entraînable).
(non-trainable) : ses poids ne seront pas mis à jour lors de l'apprentissage de gan . Si les poids du discriminateur
Si les poids du discriminateur pouvaient être mis à jour au cours de ce processus, vous entraîneriez le discriminateur à toujours prédire les " vrais " résultats.
Si les poids du discriminateur pouvaient être mis à jour au cours de ce processus, vous entraîneriez le discriminateur à toujours prédire "réel", ce qui n'est pas ce que vous voulez !


In [78]:
# Sets discriminator weights to non-trainable (this will only apply to the gan model)

discriminator.trainable = False 

gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)
gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

# Comment former votre DCGAN
Vous pouvez maintenant commencer l'entraînement. Pour récapituler, voici à quoi ressemble la boucle d'entraînement
schématiquement. Pour chaque époque, vous effectuez les opérations suivantes :
1- Dessiner des points aléatoires dans l'espace latent (bruit aléatoire).
2- Générer des images avec le générateur en utilisant ce bruit aléatoire.
3- Mélanger les images générées avec des images réelles.
4- Entraîner le discriminateur en utilisant ces images mélangées, avec les cibles correspondantes : soit "vrai" (pour les images réelles) soit "faux" (pour les images générées).
5- Dessiner de nouveaux points aléatoires dans l'espace latent.
6- Entraîner le gan en utilisant ces vecteurs aléatoires, avec des cibles qui disent toutes "ce sont de vraies images". Cela met à jour les poids du générateur (seulement, parce que le discriminateur est gelé à l'intérieur de gan) pour les faire évoluer vers la prédiction par le discriminateur de "ce sont de vraies images" pour les images générées : cela entraîne le générateur à tromper le discriminateur.
Implémentons-le.

In [40]:
dataset = keras.preprocessing.image_dataset_from_directory(
    "b17datas/test", label_mode=None, image_size=(32, 32), batch_size=32
)

# -----> Pourquoi cette ligne ?
dataset = dataset.map(lambda x: x / 255.0)


Found 500 files belonging to 1 classes.


In [41]:
dataset

<MapDataset shapes: (None, 32, 32, 3), types: tf.float32>

In [48]:
x_train = np.empty(shape=(0,height, width, 3))

for elt in dataset.as_numpy_iterator():
    x_train = np.concatenate((x_train, elt), axis=0)
      

print(x_train.shape)

(500, 32, 32, 3)


In [49]:
#import os
#
##from keras.preprocessing import image # Loads CIFAR10 data
#
#(x_train, y_train), (_, _) = keras.load_data(dataset)
#

In [50]:
print(x_train.shape)
print(type(x_train))

(500, 32, 32, 3)
<class 'numpy.ndarray'>


In [51]:
#x_train = x_train[y_train.flatten() == 6] #Selects frog images (class 6)
#
##dataset = keras.preprocessing.image_dataset_from_directory("b17datas/test", label_mode= None, image_size=(64, 64), batch_size=32)
## -----> Pourquoi cette ligne ?
##dataset = dataset.map(lambda x: x / 255.0)
#
#x_train = x_train.reshape((x_train.shape[0],) + (height, width, channels)).astype('float32') / 255. # Normalizes data
#

In [52]:
type(x_train)

numpy.ndarray

In [53]:

iterations = 1000
batch_size = 20
save_dir = 'sav_dir' # Specifies where you want to save generated images

start = 0

for step in range(iterations):
    # Samples random points in the latent space
    random_latent_vectors = np.random.normal(size=(batch_size,latent_dim))

    generated_images = generator.predict(random_latent_vectors) # Decodes them to fake images

    # 3 lignes suivantes: Combines them with real images
    stop = start + batch_size
    real_images = x_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images]) # Assembles labels, discriminating real from fake images


    labels = np.concatenate([np.ones((batch_size, 1)),
    np.zeros((batch_size, 1))])
    labels += 0.05 * np.random.random(labels.shape) # Adds random noise to the labels—an important trick!


    d_loss = discriminator.train_on_batch(combined_images, labels) # Trains the discriminator




    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim)) # Samples random points in the latent space

    misleading_targets = np.zeros((batch_size, 1)) # Assembles labels that say “these are all real images” (it’s a lie!)



    # Trains the generator (via the gan model, where the discriminator weights are frozen)
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)

    start += batch_size
    if start > len(x_train) - batch_size:
        start = 0

    if step % 100 == 0:
        print(step)

        gan.save_weights('gan.h5') # Occasionally saves and plots (every 100 steps)

        # Prints metrics
        print('discriminator loss:', d_loss)
        print('adversarial loss:', a_loss)

        # Saves one generated image
        img = image.array_to_img(generated_images[0] * 255., scale=False) 
        img.save(os.path.join(save_dir,'generated_frog' + str(step) + '.png'))

        # Saves one real image for comparison
        img = image.array_to_img(real_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir,'real_frog' + str(step) + '.png'))


0
discriminator loss: 0.6878975033760071
adversarial loss: 0.6593941450119019
100
discriminator loss: 0.5805875062942505
adversarial loss: 1.3637793064117432
200
discriminator loss: 0.6007582545280457
adversarial loss: 1.3806746006011963
300
discriminator loss: 0.6156712770462036
adversarial loss: 0.7641630172729492
400
discriminator loss: 0.5897729396820068
adversarial loss: 0.9305468797683716
500
discriminator loss: 0.5653040409088135
adversarial loss: 0.5294497013092041
600
discriminator loss: 0.48168158531188965
adversarial loss: 0.47028398513793945
700
discriminator loss: 0.5131665468215942
adversarial loss: 2.3199706077575684
800
discriminator loss: 0.5401914715766907
adversarial loss: 1.4528510570526123
900
discriminator loss: 0.5705994963645935
adversarial loss: 0.8480575680732727


## Résultats obtenue sur le code lancer depuis google colab
Le discriminateur fonctionne bien et converge, adversial par contre diverge et pourtant les images obtenues sont de meilleurs qualitées.

In [ ]:
discriminator loss: 3.460803508758545
adversarial loss: 1.1308232927831341e-07
